Bruker notebooks her ettersom koden ble relativt mye større


In [481]:
import pandas as pd
import requests

In [482]:
import pytz
from datetime import datetime

# Funksjon som endrer tiden til lokal tid, eller annen tid etter behov og formaterer den i format som gir mening
def convert_time_stamp(timestamp_str):

    utc_timezone = pytz.timezone('UTC')
    local_timezone = pytz.timezone('Europe/Copenhagen') 
    timestamp_utc = datetime.strptime(timestamp_str, '%Y-%m-%dT%H:%M:%SZ')
    timestamp_utc = utc_timezone.localize(timestamp_utc)
    timestamp_local = timestamp_utc.astimezone(local_timezone)
    local_timestamp_str = timestamp_local.strftime('%Y-%m-%d %H:%M:%S %Z')

    return local_timestamp_str

In [483]:
#Her kan du spesifisere hvilken by
city_string = "Bergen"

city = {'Oslo': (59.9333, 10.7166), 'Bergen': (60.3894, 5.33), 'Trondheim': (63.4308, 10.4034), 'Tromsø': (69.6827, 18.9427), 'Vardø': (70.3705, 31.0241)}
lat = city[city_string][0]
lon = city[city_string][1]

# velger å bruke nowcast delen av apiet for å finne den nyeste mest relevante informasjonen om været
url = f"https://api.met.no/weatherapi/nowcast/2.0/complete?lat={lat}&lon={lon}"
headers = {"User-Agent": "WeatherTest/1.0 skybert@gmail.com"}

response = requests.get(url, headers=headers)
result = {}

if response.status_code == 200:
    result = response.json()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [484]:
# Lage variabler med dictonaries med relevant informasjon

units_dict = result["properties"]["meta"]["units"]
units_measured_list = result["properties"]["timeseries"]


In [485]:
# Velger å bruke pandas dataframes, greit om datamengdene er store og om mye transformering må gjøres på mye data så er det raskere i pandas enn i python. Skalerbart

unit_df = pd.DataFrame.from_dict(units_dict, orient='index', columns=['Unit'])
unit_df.index.name = 'Attribute'

In [486]:
raw_data_df = pd.DataFrame()

# blar igjennom listen med dictionaries og filtrer ut det vi vil ha og legger det i en dataframe
for i in units_measured_list:
    i_details = i["data"]["instant"]["details"]
    i_timestamp = i["time"]

    #next 1 hours forekommer bare i første tilfelle, og ikke resten av nedbørsmengden, så vi trenger if statement for å forhindre crash når den ikke fins
    if 'next_1_hours' in i['data']:
        next_1_hours = i["data"]["next_1_hours"]["summary"]["symbol_code"].replace("_", " ")
    else:
        next_1_hours = None
    i_details["next_1_hour"] = next_1_hours

    raw_data_df[i_timestamp] = i_details

In [487]:
#gi navn på kolonner med tidspunkt som gir mening
raw_data_df.columns = raw_data_df.columns.map(convert_time_stamp)


In [488]:
# Gi navn på kolonne slik at det blir lettere å se hva som merges isteden for å bruke indexer
raw_data_df.index.name = 'Attribute'

In [489]:
# her spesifiseres hva som skal hentes ut, i dette tilfellet alt
features = ['air_temperature', 'precipitation_rate', 'relative_humidity', 'wind_from_direction', 'wind_speed',"next_1_hour"]
selected_rows = raw_data_df.loc[features]
precipitation_rate_df = pd.DataFrame(raw_data_df.loc["precipitation_rate"])

In [490]:
#Fjerner kolonner med mange nan verdier, med dette datasettet er det kun precipitation_rate som har større tidsrom på målinger
selected_rows = selected_rows.dropna(axis=1, how='any')

In [491]:
# Her kombinerer vi inn dataen med hvilke målenheter det er
combined_df = selected_rows.join(unit_df, how='left')

# I dette tilfellet har vi bare 1 NaN verdi som til hører cloud, og vi setter den til å være weather som gir mest mening selv om den egentlig ikke er en målig
combined_df = combined_df.fillna('weather')
combined_df.index = combined_df.index.str.replace('_', " ")

In [492]:
print(f"Current weather for: {city_string} ")
print(f"Measured on: {combined_df.columns[0]} ")
print(f"Weather for lat: {lat},lon: {lon}:")

# Iterer gjennom dataframen og printer all dataen
for index, row in combined_df.iterrows():
    print(f"{index}: {row[combined_df.columns[0]]} {row[combined_df.columns[1]]}")

# Om det er noe nedbør så printer den bare når det er nedbør
if (precipitation_rate_df.iloc[:, 0] > 0).any():
    print("Times with precipitation:")
    for index, row in precipitation_rate_df.iterrows():
        if (row[precipitation_rate_df.columns[0]] > 0):
            #bruker split()[1] her bare for å få ut tidspunktet
            print(f"{index.split()[1]}: {row[precipitation_rate_df.columns[0]]}")
else:
    print("No further precipitation")

Current weather for: Bergen 
Measured on: 2024-02-13 20:05:00 CET 
Weather for lat: 60.3894,lon: 5.33:
air temperature: 3.8 celsius
precipitation rate: 0.3 mm/h
relative humidity: 91.6 %
wind from direction: 153.2 degrees
wind speed: 6.5 m/s
next 1 hour: rain weather
Times with precipitation:
20:05:00: 0.3
20:10:00: 0.7
20:15:00: 0.9
20:20:00: 0.2
20:30:00: 0.2
20:35:00: 1.9
20:40:00: 2.8
20:45:00: 2.4
20:50:00: 1.2
20:55:00: 0.4
21:00:00: 0.2
21:05:00: 0.3
21:10:00: 0.4
21:15:00: 0.7
21:20:00: 1.0
21:25:00: 1.8
21:30:00: 2.0
21:35:00: 2.1
21:40:00: 1.8
21:45:00: 0.9


```
Eksempel fra output fra 13-02 kl 20:00:
Current weather for: Bergen 
Measured on: 2024-02-13 20:00:00 CET 
Weather for lat: 60.3894,lon: 5.33:
air temperature: 3.7 celsius
precipitation rate: 0.0 mm/h
relative humidity: 91.7 %
wind from direction: 153.0 degrees
wind speed: 6.5 m/s
next 1 hour: rain weather
Times with precipitation:
20:05:00: 0.2
20:10:00: 0.3
20:15:00: 0.7
20:20:00: 0.2
20:30:00: 0.2
20:35:00: 1.3
20:40:00: 1.7
20:45:00: 1.3
20:50:00: 0.4
20:55:00: 0.2
21:00:00: 0.1
21:05:00: 0.2
21:10:00: 0.3
21:15:00: 0.5
21:20:00: 0.6
21:25:00: 0.4```